In [1]:
import datetime
import glob
import os

import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import scipy.stats as stats
import tensorflow as tf
from tqdm import tqdm

import utils

In [3]:
chour_dict = {'morning':'0400',
              'afternoon':'1200',
             }

ampm = 'morning'

width = 16

critical_hour = chour_dict[ampm]

df = pd.read_csv(f'merged_df_{ampm}.csv',parse_dates=['time'])
time = df['time']
df = df.drop(['time','liq_prec','frz_prec','tmpf','speed','speed_diff'],axis=1)

In [4]:
(train_df,val_df,test_df), num_features = utils.train_test_val_split(df)

In [9]:
day_list = []
for group in df.groupby(time.dt.date):
    if ((group[1]['frz_prec_pred']>0)).any():
        day_list.append(group[1])

In [10]:
len(day_list)

22

In [11]:
lstm_no = tf.keras.models.load_model(f'no_weather_{ampm}')
lstm_wx = tf.keras.models.load_model(f'weather_{ampm}_test')

In [12]:
def z_to_anom(value,name,ampm):
    std = pd.read_csv(f'std_{ampm}.csv',index_col=0)
    std = std.T[name].values
    return (value*std)

def z_to_value(value,name,ampm):
    mn = pd.read_csv(f'mean_{ampm}.csv',index_col=0)
    mn = mn.T[name].values
    std = pd.read_csv(f'std_{ampm}.csv',index_col=0)
    std = std.T[name].values
    return (value*std)+mn

def norm(value,name,ampm):
    mn = pd.read_csv(f'mean_{ampm}.csv',index_col=0)
    mn = mn.T[name].values
    std = pd.read_csv(f'std_{ampm}.csv',index_col=0)
    std = std.T[name].values
    return (value-mn)/std

In [18]:
df[df['frz_prec_pred']>0].corr()

,frz_prec_pred,liq_prec_pred,hour,speed_anom
frz_prec_pred,1.000000,0.089419,-0.256188,-0.381718
liq_prec_pred,0.089419,1.000000,0.088002,0.228112
hour,-0.256188,0.088002,1.000000,-0.058548
speed_anom,-0.381718,0.228112,-0.058548,1.000000


In [17]:
df[df['liq_prec_pred']>0].corr()

,frz_prec_pred,liq_prec_pred,hour,speed_anom
frz_prec_pred,1.000000,0.025666,0.023509,0.058964
liq_prec_pred,0.025666,1.000000,-0.057271,-0.212249
hour,0.023509,-0.057271,1.000000,-0.043850
speed_anom,0.058964,-0.212249,-0.043850,1.000000


In [19]:
tem = test_df[(test_df['liq_prec_pred']>0)]

In [20]:
time_test = time[tem.index]

In [21]:
forecast_starts = pd.date_range(f'2015-01-01 {critical_hour}',f'2019-12-31 {critical_hour}',freq='24h')
# valid_inds = time_test[time_test.isin(forecast_starts)].index
valid_inds = time_test[(time_test.dt.hour<10) & (time_test.dt.hour>4)].index

In [24]:
rmse = np.ones(len(valid_inds))
val = np.ones((len(valid_inds),8))*np.nan
pred = np.ones((len(valid_inds),8))*np.nan


for ind_val,ind in enumerate(valid_inds):
        pred_i = lstm_wx.predict(df[ind:ind+width].values[np.newaxis,])
        pred_i = pred_i[0,:,1]
        val_i = df[ind+width:ind+2*width]['speed_anom']
        if len(val_i) > 7:
            rmse[ind_val] = np.sqrt(((val_i.values-pred_i.T)**2).mean())

print(rmse.mean())
# print(no_norm(rmse.mean(),'speed_anom',mn,std))

0.5555605652238194


In [27]:
df_no = df.drop(['liq_prec_pred','frz_prec_pred'],axis=1)

In [28]:
def no_norm(value,name,mn,std):
    return (value*std)[name].values[0]

In [30]:
rmse = np.ones(len(valid_inds))
val = np.ones((len(valid_inds),8))*np.nan
pred = np.ones((len(valid_inds),8))*np.nan

for ind_val,ind in enumerate(valid_inds):
        pred_i = lstm_no.predict(df_no[ind:ind+width].values[np.newaxis,])
        pred_i = pred_i[0,:,1]
        val_i = df_no[ind+width:ind+2*width]['speed_anom']
        if len(val_i) > 7:
            rmse[ind_val] = np.sqrt(((val_i.values-pred_i.T)**2).mean())
print(rmse.mean())

0.5675943700569179
